In [9]:
import xarray as xr
import geopandas as gpd
import pandas as pd
from dask.diagnostics import ProgressBar

# Read in the UTCI data

This is 3 hourly historical data, and will be used for training the mortality curves

In [10]:
%pwd

'/home/users/train026/repo/project10/notebooks'

In [5]:
dataset =  xr.open_mfdataset('/gws/pw/j05/cop26_hackathons/bristol/project10/utci_projections_1deg/HadGEM3-GC31-LL/historical/r1i1p1f3/*.nc')
dataset

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 180, lon: 360, time: 86400)
Coordinates:
  * time      (time) object 1985-01-01 03:00:00 ... 2015-01-01 00:00:00
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables:
    utci      (time, lat, lon) float64 dask.array<chunksize=(2880, 180, 360), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(2880, 180, 2), meta=np.ndarray>
    lon_bnds  (time, lon, bnds) float64 dask.array<chunksize=(2880, 360, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

# Read in shapefiles for city locations

In [11]:
shps = gpd.read_file("../data/raw_data/ne_10m_populated_places_simple/ne_10m_populated_places_simple.shp")
print(shps)

      scalerank  natscale  labelrank              featurecla  \
0            10         1          8         Admin-1 capital   
1            10         1          8         Admin-1 capital   
2            10         1          8         Admin-1 capital   
3            10         1          8         Admin-1 capital   
4            10         1          8         Admin-1 capital   
...         ...       ...        ...                     ...   
7338          0       600          1         Admin-1 capital   
7339          0       600          1         Admin-1 capital   
7340          0       600          3         Admin-1 capital   
7341          0       600          0         Admin-0 capital   
7342          0       600          0  Admin-0 region capital   

                        name namepar              namealt  diffascii  \
0     Colonia del Sacramento    None                 None          0   
1                   Trinidad    None                 None          0   
2              

In [75]:
# Alternative approach to download directly from site

# from urllib.request import urlopen
# from io import BytesIO
# from zipfile import ZipFile


# def download_and_unzip(url, extract_to='.'):
#     http_response = urlopen(url)
#     zipfile = ZipFile(BytesIO(http_response.read()))
#     zipfile.extractall(path=extract_to)

# download_and_unzip("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip")
# shp = gpd.read_file("ne_10m_populated_places.shp")
# print(shp)

In [13]:
shps.columns

Index(['scalerank', 'natscale', 'labelrank', 'featurecla', 'name', 'namepar',
       'namealt', 'diffascii', 'nameascii', 'adm0cap', 'capalt', 'capin',
       'worldcity', 'megacity', 'sov0name', 'sov_a3', 'adm0name', 'adm0_a3',
       'adm1name', 'iso_a2', 'note', 'latitude', 'longitude', 'changed',
       'namediff', 'diffnote', 'pop_max', 'pop_min', 'pop_other', 'rank_max',
       'rank_min', 'geonameid', 'meganame', 'ls_name', 'ls_match', 'checkme',
       'min_zoom', 'ne_id', 'geometry'],
      dtype='object')

In [45]:
(shps.query('name in ["Alofi", "Florida"]'))

,scalerank,natscale,labelrank,featurecla,name,namepar,namealt,diffascii,nameascii,adm0cap,...,rank_max,rank_min,geonameid,meganame,ls_name,ls_match,checkme,min_zoom,ne_id,geometry
4,10,1,8,Admin-1 capital,Florida,None,None,0,Florida,0.0,...,7,7,3442585.0,None,None,0,0,7.0,1159112703,POINT (-56.21500 -34.09900)
1812,8,10,8,Admin-1 capital,Alofi,None,None,0,Alofi,0.0,...,0,0,4036284.0,None,None,0,0,7.0,1159151741,POINT (-169.91363 -19.06599)


In [14]:
shps.groupby(['adm0name']).agg('size')

adm0name
Afghanistan       33
Aland              1
Albania           26
Algeria           51
American Samoa     1
                  ..
Vietnam           60
Western Sahara     1
Yemen             20
Zambia            34
Zimbabwe          20
Length: 228, dtype: int64

In [15]:
populated = (shps
    .assign(
        lon=shps['geometry'].x,
        lat=shps['geometry'].y,
        pop=(shps['pop_max']+shps['pop_min'])/2)
    .filter(['name', 'adm0name', 'pop', 'lon', 'lat']))
populated

,name,adm0name,pop,lon,lat
0,Colonia del Sacramento,Uruguay,21714.0,-57.840002,-34.479999
1,Trinidad,Uruguay,21093.0,-56.900997,-33.543999
2,Fray Bentos,Uruguay,23279.0,-58.303997,-33.138999
3,Canelones,Uruguay,19698.0,-56.284001,-34.538004
4,Florida,Uruguay,32234.0,-56.214998,-34.099002
...,...,...,...,...,...
7338,Rio de Janeiro,Brazil,6879087.5,-43.226967,-22.923077
7339,São Paulo,Brazil,14433147.5,-46.626966,-23.556734
7340,Sydney,Australia,4135711.0,151.183234,-33.918065
7341,Singapore,Singapore,4236614.5,103.853875,1.294979


populated.to_parquet('../data/processed_data/populated.parquet.gz', compression='gzip')

In [16]:
france = populated.query('adm0name == "France"').reset_index()
france.to_parquet('../data/processed_data/france.parquet.gz', compression='gzip')

<ipython-input-16-0662050d4dd4>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  france.to_parquet('../data/processed_data/france.parquet.gz', compression='gzip')


# Read in the UTCI data

This is 3 hourly historical data, and will be used for training the mortality curves

In [17]:
utci =  xr.open_mfdataset('/gws/pw/j05/cop26_hackathons/bristol/project10/utci_projections_1deg/HadGEM3-GC31-LL/historical/r1i1p1f3/*.nc', parallel=True)
utci

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 180, lon: 360, time: 86400)
Coordinates:
  * time      (time) object 1985-01-01 03:00:00 ... 2015-01-01 00:00:00
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables:
    utci      (time, lat, lon) float64 dask.array<chunksize=(2880, 180, 360), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(2880, 180, 2), meta=np.ndarray>
    lon_bnds  (time, lon, bnds) float64 dask.array<chunksize=(2880, 360, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

Subset to only have UK locations

In [69]:
# For the xarray object
# - select the coordinates for a given city
# - calculate the monthly mean UTCI at those coordinates
# - return a dataframe with utci over time, as well as the city info
def mean_utci_position(utci, pop, row):
    with ProgressBar():
        out = (utci
               .sel(lon=pop['lon'][row], lat=pop['lat'][row], method='nearest')['utci']
               .resample(time="1M")
               .mean()
               .compute()
               .to_dataframe()
               .assign(
                   name=pop['name'][row],
                   adm0name=pop['adm0name'][row],
                   pop=pop['pop'][row],
                   lon_orig=pop['lon'][row],
                   lat_orig=pop['lat'][row]               
               ))
        out = (out
            .assign(
                year = [x.year for x in out.index],
                month = [x.month for x in out.index]
            )
            .reset_index(drop=True))
    return(out)

In [19]:
def country_utci(utci, populated, country):
    sub = populated.query('adm0name == "{}"'.format(country)).reset_index()
    print("Calculating " + str(sub.shape[0]) + " regions")
    out = [mean_utci_position(utci, sub, row) for row in range(sub.shape[0])]
    return(pd.concat(out))

In [40]:
a = populated.groupby('adm0name').agg({'adm0name':['size']})
a.columns = a.columns.droplevel(0)
a.query('size == 2')

,size
adm0name,
Cape Verde,2
Falkland Islands,2
Faroe Islands,2
Mauritius,2
Northern Cyprus,2
Palau,2
Sao Tome and Principe,2
Slovenia,2
The Bahamas,2


In [76]:
out = country_utci(utci, populated, "Slovenia")

Calculating 2 regions
[########################################] | 100% Completed |  9min 27.1s
[########################################] | 100% Completed |  2.7s


In [145]:
out = out.assign(model="HadGEM3-GC31-LL", scenario="historical")

In [135]:
popsum = out.groupby('name').apply(lambda x: x['pop'].loc[0]).sum()
o = (out
     .assign(utci=out['pop'] * out['utci']/popsum)
     .groupby(['year', 'month', 'adm0name'])
     .agg({'utci':['sum']})
     .reset_index()
)
o.columns = o.columns.droplevel(1)
o

,year,month,adm0name,utci
0,1985,1,Slovenia,267.690702
1,1985,2,Slovenia,274.835998
2,1985,3,Slovenia,277.477339
3,1985,4,Slovenia,282.735179
4,1985,5,Slovenia,290.590610
...,...,...,...,...
356,2014,9,Slovenia,290.748214
357,2014,10,Slovenia,285.535082
358,2014,11,Slovenia,273.813229
359,2014,12,Slovenia,269.916172


In [71]:
out = country_utci(utci, populated, "Monaco")
out = out.assign(model="HadGEM3-GC31-LL", scenario="historical")
out
out.to_parquet("../data/processed_data/utci_country_monthly/monaco_HadGEM3-GC31-LL_historical.parquet.gz", compression="gzip")

,lat,lon,utci,name,adm0name,pop,lon_orig,lat_orig,year,month,model,scenario
0,43.5,7.5,273.848787,Monaco,Monaco,36371.0,7.406913,43.739646,1985,1,HadGEM3-GC31-LL,historical
1,43.5,7.5,282.069428,Monaco,Monaco,36371.0,7.406913,43.739646,1985,2,HadGEM3-GC31-LL,historical
2,43.5,7.5,281.880166,Monaco,Monaco,36371.0,7.406913,43.739646,1985,3,HadGEM3-GC31-LL,historical
3,43.5,7.5,285.968089,Monaco,Monaco,36371.0,7.406913,43.739646,1985,4,HadGEM3-GC31-LL,historical
4,43.5,7.5,292.263468,Monaco,Monaco,36371.0,7.406913,43.739646,1985,5,HadGEM3-GC31-LL,historical
...,...,...,...,...,...,...,...,...,...,...,...,...
356,43.5,7.5,295.759083,Monaco,Monaco,36371.0,7.406913,43.739646,2014,9,HadGEM3-GC31-LL,historical
357,43.5,7.5,290.545414,Monaco,Monaco,36371.0,7.406913,43.739646,2014,10,HadGEM3-GC31-LL,historical
358,43.5,7.5,281.987321,Monaco,Monaco,36371.0,7.406913,43.739646,2014,11,HadGEM3-GC31-LL,historical
359,43.5,7.5,279.339864,Monaco,Monaco,36371.0,7.406913,43.739646,2014,12,HadGEM3-GC31-LL,historical


Aggregate country, weighted by region populations

In [129]:
def aggregate_country(out):
    popsum = out.groupby('name').apply(lambda x: x['pop'].loc[0]).sum()
    o = (out
         .assign(utci=out['pop'] * out['utci']/popsum)
         .groupby(['year', 'month', 'adm0name', 'model', 'scenario'])
         .agg({'utci':['sum']})
         .reset_index()
    )
    o.columns = o.columns.droplevel(1)
    return(o)

In [147]:
aggregate_country(out)

,year,month,adm0name,model,scenario,utci
0,1985,1,Slovenia,HadGEM3-GC31-LL,historical,267.690702
1,1985,2,Slovenia,HadGEM3-GC31-LL,historical,274.835998
2,1985,3,Slovenia,HadGEM3-GC31-LL,historical,277.477339
3,1985,4,Slovenia,HadGEM3-GC31-LL,historical,282.735179
4,1985,5,Slovenia,HadGEM3-GC31-LL,historical,290.590610
...,...,...,...,...,...,...
356,2014,9,Slovenia,HadGEM3-GC31-LL,historical,290.748214
357,2014,10,Slovenia,HadGEM3-GC31-LL,historical,285.535082
358,2014,11,Slovenia,HadGEM3-GC31-LL,historical,273.813229
359,2014,12,Slovenia,HadGEM3-GC31-LL,historical,269.916172


In [148]:
out

,lat,lon,utci,name,adm0name,pop,lon_orig,lat_orig,year,month,model,scenario
0,46.5,15.5,267.350397,Maribor,Slovenia,101642.0,15.650042,46.540478,1985,1,HadGEM3-GC31-LL,historical
1,46.5,15.5,274.200120,Maribor,Slovenia,101642.0,15.650042,46.540478,1985,2,HadGEM3-GC31-LL,historical
2,46.5,15.5,276.565309,Maribor,Slovenia,101642.0,15.650042,46.540478,1985,3,HadGEM3-GC31-LL,historical
3,46.5,15.5,282.549803,Maribor,Slovenia,101642.0,15.650042,46.540478,1985,4,HadGEM3-GC31-LL,historical
4,46.5,15.5,291.093620,Maribor,Slovenia,101642.0,15.650042,46.540478,1985,5,HadGEM3-GC31-LL,historical
...,...,...,...,...,...,...,...,...,...,...,...,...
356,46.5,14.5,290.655382,Ljubljana,Slovenia,284961.0,14.514969,46.055288,2014,9,HadGEM3-GC31-LL,historical
357,46.5,14.5,285.477926,Ljubljana,Slovenia,284961.0,14.514969,46.055288,2014,10,HadGEM3-GC31-LL,historical
358,46.5,14.5,273.887317,Ljubljana,Slovenia,284961.0,14.514969,46.055288,2014,11,HadGEM3-GC31-LL,historical
359,46.5,14.5,270.035762,Ljubljana,Slovenia,284961.0,14.514969,46.055288,2014,12,HadGEM3-GC31-LL,historical
